In [3]:
import warnings
warnings.filterwarnings("ignore")
import datetime
import numpy as np
import pandas as pd
import pickle
import xarray as xr
get_ipython().run_line_magic('matplotlib', 'inline')
from scipy.stats import pearsonr

import pyPamtra
import Regression_Retrieval
import PAMTRA_sim_analysis

import cartopy.crs as ccrs
import matplotlib.pyplot as plt

### Path definitions

In [4]:
import os 
plot_path=os.getcwd()+"/plots/"
take_all_days=True
take_testing=False
take_random_days=False
take_snd_random_days=False
take_haloac3_day=False
ar_plot_path=plot_path+"synth_ARs/"
if take_testing:
    ar_plot_path=plot_path+"testing_cases/"
else:
    if take_random_days:
        ar_plot_path=plot_path+"rand_days/"
        if take_snd_random_days:
            ar_plot_path=plot_path+"snd_rand_days/"
    if take_haloac3_day:
        ar_plot_path=ar_plot_path+"/../halo_ac3/"
    if take_all_days:
        ar_plot_path=plot_path+"new_all_days/"
    if not os.path.exists(plot_path):
        os.makedirs(plot_path)
if not os.path.exists(ar_plot_path):
    os.makedirs(ar_plot_path)
    print(ar_plot_path)
hour="12"
obs_height=12000

In [5]:
var_to_retrieve="q"
# Dataset Training sample
#synth_ar_dates=["20110317","20110423","20150314",
#                "20160311","20180224",
#                "20180225","20190319",
#                "20200416","20200419"
#                   ]
synth_ar_dates=['19790312', '19810330', '19810424',# '19820320', '19820416',# '19830316', '19830331', '19830414', 
    #'19840308', '19840413', '19840428', '19880330', '19890422', '19900409', '19900411', '19900418', 
    #'19910301', '19910428', '19920302',
    #'19920413', '19930303', '19930430', '19940421', '19950306', 
    #'19950317', '19970322', '19980324', '19980409', '20030415', '20050401', '20060317', '20070324',
    #'20080313', '20080404', '20080411', '20080430', '20090324', '20100323', '20110311', '20110329', 
    #'20110413', '20140309', '20140330', '20160409', '20180319', '20180326', '20200326', 
    
                #            '20220315', 
    #'20220316', '20220410'
    ]
if take_all_days:
    PAMTRASIM_analysis=PAMTRA_sim_analysis.PAMTRASIM_analysis
    synth_ar_dates=PAMTRASIM_analysis.list_all_simulated_days(hour=hour)
if take_haloac3_day:
    synth_ar_dates=["20220315"]
# add noise for TBs
add_noise=True

regridded_era5_ds_dict={}


51  days are already simulated
['19790312', '19810330', '19810424', '19820320', '19820416', '19830316', '19830331', '19830414', '19840308', '19840413', '19840428', '19880330', '19890422', '19900409', '19900411', '19900418', '19910301', '19910428', '19920302', '19920413', '19930303', '19930430', '19940421', '19950306', '19950317', '19970322', '19980324', '19980409', '20030415', '20050401', '20060317', '20070324', '20080313', '20080404', '20080411', '20080430', '20090324', '20100323', '20110311', '20110329', '20110413', '20120312', '20130313', '20140309', '20140330', '20160409', '20180319', '20180326', '20200326', '20210402', '20220310']


In [6]:
def merge_era5_quantities(synth_ar_dates):
    for d,date in enumerate(synth_ar_dates):
        print(date)
        sim_cls=sim_cls=PAMTRA_sim_analysis.PAMTRASIM_analysis(date=date,hour=hour)
        sim_cls.open_era5_ocean_fields()
    
        #Regrid ERA5 to unified grid
        #interpolate ERA5 data onto regular height grid
        sim_cls.create_regridded_era5()
        ### Calc specific humidity q for regridded era5
        sim_cls.regridded_era5=sim_cls.calc_specific_humidity_from_relative_humidity(sim_cls.regridded_era5)
        sim_cls.regridded_era5["idx"]=sim_cls.regridded_era5["idx"]+int(date)*1000
        regridded_era5_ds_dict[date]=sim_cls.regridded_era5
    merged_regridded_era5=xr.concat([*regridded_era5_ds_dict.values()],dim="idx")
    return merged_regridded_era5

def get_merged_tbs_at_obs_height(synth_ar_dates,height):
    PAMTRASIM_analysis=PAMTRA_sim_analysis.PAMTRASIM_analysis
    
    for d,date_to_analyse in enumerate(synth_ar_dates):
        print(date_to_analyse)
        tmp_sim_cls=PAMTRASIM_analysis(date=date_to_analyse,hour=hour)    
        tmp_sim_cls.open_pamtra_tbs_ocean()
        output_heights=pd.Series(data=np.array(tmp_sim_cls.pamtra_ds.obs_height.values))
        relevant_height_idx=int(output_heights.index[output_heights == height].values[0])#.tolist()output_heights.get_loc()
        tmp_sim_cls.tb_da=tmp_sim_cls.tb_da.isel({"nout":relevant_height_idx})
        temp_df=pd.DataFrame(data=tmp_sim_cls.tb_da.values,
                         columns=tmp_sim_cls.tb_da.freq)
        temp_df.index=temp_df.index+int(date_to_analyse)*1000 
        temp_df["Date"]=date_to_analyse
        if d==0:
            tb_df=temp_df
        else:
            tb_df=pd.concat([tb_df,temp_df])

    del tb_df[182.71]
    return tb_df,tmp_sim_cls

def add_noise_to_synth_TBs(tb_df):
    # add noise to PAMTRA TBs
    # create randomn vector
    # k,v band noise 0:14
    for kv_band_freq in range(14):
        random_noise=np.random.random_sample(tb_df.shape[0])
        values=pd.Series(data=-0.25 + 0.5*random_noise,index=tb_df.index)
        tb_df.iloc[:,kv_band_freq]=tb_df.iloc[:,kv_band_freq]+values
    
    for other_freq in np.arange(14,int(tb_df.shape[1]-1)):
        random_noise=np.random.random_sample(tb_df.shape[0])
        values=pd.Series(data=-0.5 + 1.0*random_noise,index=tb_df.index)
        tb_df.iloc[:,other_freq]=tb_df.iloc[:,other_freq]+values

    return tb_df
        #pass

### Regrid ERA5 to unified grid

In [7]:
merged_regridded_era5=merge_era5_quantities(synth_ar_dates)

if var_to_retrieve=="log_q":
    merged_regridded_era5["log_q"]=np.log(merged_regridded_era5["q"])
merged_regridded_era5

19790312
Regrid  t
[####################] 100% Regrid  p
[####################] 100% Regrid  rh
[####################] 100% mixing_ratio calculated
specific humidity calculated
19810330
Regrid  t
[####################] 100% Regrid  p
[####################] 100% Regrid  rh
[####################] 100% mixing_ratio calculated
specific humidity calculated
19810424
Regrid  t
[####################] 100% Regrid  p
[####################] 100% Regrid  rh
[####################] 100% mixing_ratio calculated
specific humidity calculated
19820320
Regrid  t
[####################] 100% Regrid  p
[####################] 100% Regrid  rh
[####################] 100% mixing_ratio calculated
specific humidity calculated
19820416
Regrid  t
[####################] 100% Regrid  p
[####################] 100% Regrid  rh
[####################] 100% mixing_ratio calculated
specific humidity calculated
19830316
Regrid  t
[####################] 100% Regrid  p
[####################] 100% Regrid  rh
[##################

<xarray.Dataset>
Dimensions:  (z: 534, idx: 24265)
Coordinates:
  * z        (z) float64 0.0 30.0 60.0 90.0 ... 1.593e+04 1.596e+04 1.599e+04
  * idx      (idx) int64 19790312000 19790312001 ... 20220310521 20220310522
Data variables:
    lat      (idx) float64 65.34 65.34 65.34 65.34 ... 74.33 73.21 72.08 70.96
    lon      (idx) float64 -29.81 -28.69 -27.56 -26.44 ... 48.94 48.94 48.94
    t        (idx, z) float64 270.0 269.4 269.0 268.7 ... 194.6 194.6 194.6 nan
    p        (idx, z) float64 1.015e+05 1.011e+05 1.008e+05 ... 9.049e+03 nan
    rh       (idx, z) float64 60.14 56.49 56.23 56.38 ... 31.42 31.46 31.51 nan
    q        (idx, z) float32 0.001796 0.00162 0.001568 ... 2.913e-06 nan

### Get PAMTRA-HAMP TBs

In [8]:
height=12000
obs_heights=[16000., 15400.,15200.,15000.,14800.,14600.,14400.,14200.,14000.,13800.,13600.,
            13400,13200,13000,12800,12600,12400,12200,12000., 11800.,11600.,11400.,
            11200.,11000.,10800.,10600.,10400.,10200.,10000.,9800.,9600.,9400.,9200.,
            9000., 8800.,8600.,8400.,8200.,8000.,7800.,7600.,7400.,7200.,7000.]
#utput_heights=[16000, 15500,15000,14500,14000,13000,12500,12000.,11900., 11800., 11700., 11600., 11500., 
            #           11400., 11300., 11200.,11100., 11000.,10900., 10800., 10700., 10600., 10500., 10400.,10300.,
            #           10200., 10100., 10000., 9900.,  9800.,  9700.,  9600., 9500.,  9400.,  9300.,  9200.,  9100.,
            #           9000., 8900., 8800.,8700., 8600., 8500.,#8400,8300,8200,8100,8000,
            #           0.]#[12000,11000,10000]
for height in obs_heights:
    tb_df,sim_cls=get_merged_tbs_at_obs_height(synth_ar_dates,height)
    if add_noise:
        print("add noise to bright band temperatures")
        tb_df=add_noise_to_synth_TBs(tb_df)
    else:
        print("Caution: No noise added to PAMTRA TBs")
    
    #Vertical Regression Retrieval
    retrieval=Regression_Retrieval.Regression_Retrieval
    order=2
    brightness_Ts=tb_df.copy()
    #sim_cls.regridded_era5
    if len(synth_ar_dates)>1:
        era5_ds=merged_regridded_era5
    else:
        era5_ds=sim_cls.regridded_era5
    Retrieval=retrieval(era5_ds,var_to_retrieve,brightness_Ts,order,synth_ar_dates,ar_plot_path,added_noise=True,
                       obs_height=height)
    Retrieval.get_vertical_m_est(era5_ds)

19790312
19810330
19810424
19820320
19820416
19830316
19830331
19830414
19840308
19840413
19840428
19880330
19890422
19900409
19900411
19900418
19910301
19910428
19920302
19920413
19930303
19930430
19940421
19950306
19950317
19970322
19980324
19980409
20030415
20050401
20060317
20070324
20080313
20080404
20080411
20080430
20090324
20100323
20110311
20110329
20110413
20120312
20130313
20140309
20140330
20160409
20180319
20180326
20200326
20210402
20220310
add noise to bright band temperatures
Retrieve  q  via Regression
Height Regression
[####################] 100% Retrieval coeffs saved as: /home/u/u300737/era5_hamp_pamtra/plots/new_all_days/q_Retrieval_coeffs_several_dates_16000.0.csv
19790312
19810330
19810424
19820320
19820416
19830316
19830331
19830414
19840308
19840413
19840428
19880330
19890422
19900409
19900411
19900418
19910301
19910428
19920302
19920413
19930303
19930430
19940421
19950306
19950317
19970322
19980324
19980409
20030415
20050401
20060317
20070324
20080313
20080404

## Vertical Regression Retrieval

In [9]:
Retrieval.height_m_est
#brightness_Ts["Date"].unique()

,error,22.24_a*1,23.04_a*1,23.84_a*1,25.44_a*1,26.24_a*1,27.84_a*1,31.4_a*1,50.3_a*1,51.76_a*1,...,120.15_b**2,121.05_b**2,122.95_b**2,127.25_b**2,183.91_b**2,184.81_b**2,185.81_b**2,186.81_b**2,188.31_b**2,190.81_b**2
0.0,0.028278,0.000454,0.000544,-0.001108,-0.000796,-0.000822,0.000616,0.001376,-0.003473,0.001895,...,0.000003,7.840286e-07,-0.000001,-0.000003,-0.000002,7.044010e-07,0.000002,-6.910917e-08,0.000002,0.000001
30.0,-0.001489,0.000499,0.000536,-0.001332,-0.000876,-0.000780,0.000822,0.001248,-0.003392,0.001750,...,0.000003,1.299726e-06,-0.000001,-0.000003,-0.000002,8.189133e-07,0.000002,-2.950646e-08,0.000002,0.000001
60.0,-0.009760,0.000522,0.000474,-0.001345,-0.000830,-0.000700,0.000824,0.001151,-0.003262,0.001739,...,0.000003,1.441829e-06,-0.000001,-0.000003,-0.000002,9.594833e-07,0.000002,2.797140e-08,0.000002,0.000001
90.0,-0.022113,0.000531,0.000415,-0.001373,-0.000765,-0.000648,0.000875,0.001072,-0.003119,0.001539,...,0.000003,1.709224e-06,-0.000001,-0.000004,-0.000003,9.843885e-07,0.000003,1.134768e-07,0.000002,0.000001
120.0,-0.032353,0.000529,0.000375,-0.001411,-0.000704,-0.000575,0.000901,0.001007,-0.002971,0.001280,...,0.000003,2.022486e-06,-0.000001,-0.000004,-0.000003,1.000801e-06,0.000003,2.263485e-07,0.000002,0.000001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15870.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15900.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15930.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15960.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
date="19890422"#20150314"
test_sim_cls=sim_cls=PAMTRA_sim_analysis.PAMTRASIM_analysis(date=date,hour="12")
test_sim_cls.open_era5_ocean_fields()
test_era5_ds=test_sim_cls.era5_ds
test_sim_cls.create_regridded_era5()
regridded_test_era5=sim_cls.regridded_era5
regridded_test_era5=sim_cls.calc_specific_humidity_from_relative_humidity(regridded_test_era5)
test_sim_cls.regridded_era5=regridded_test_era5
regridded_test_era5

Regrid  t
[####################] 100% Regrid  p
[####################] 100% Regrid  rh
[####################] 100% mixing_ratio calculated
specific humidity calculated


<xarray.Dataset>
Dimensions:  (idx: 464, z: 534)
Coordinates:
  * idx      (idx) int64 0 1 2 3 4 5 6 7 8 ... 456 457 458 459 460 461 462 463
  * z        (z) float64 0.0 30.0 60.0 90.0 ... 1.593e+04 1.596e+04 1.599e+04
Data variables:
    lat      (idx) float64 65.34 65.34 65.34 65.34 ... 74.33 73.21 72.08 70.96
    lon      (idx) float64 -29.81 -28.69 -27.56 -26.44 ... 48.94 48.94 48.94
    t        (idx, z) float64 274.9 274.6 274.3 274.1 ... 222.6 222.5 222.5
    p        (idx, z) float64 1.039e+05 1.035e+05 ... 1.05e+04 1.045e+04
    rh       (idx, z) float64 68.56 69.1 69.87 70.81 ... 0.6998 0.7006 0.7014
    q        (idx, z) float32 0.002856 0.002825 0.002812 ... 2.463e-06 2.466e-06

### Test run

In [11]:
test_sim_cls.open_pamtra_tbs_ocean()
temp_df=pd.DataFrame(data=test_sim_cls.tb_da.values,columns=test_sim_cls.tb_da.freq)
test_TBs=temp_df.iloc[200:202,:]
test_TBs["Date"]=date
del test_TBs[182.71]
## Built K
if var_to_retrieve=="log_q":
    regridded_test_era5["log_q"]=np.log(regridded_test_era5["q"])
    state_vector=regridded_test_era5["log_q"][200:202,:]
else:
    state_vector=regridded_test_era5[var_to_retrieve][200:202,:]*1000
state_vector
Retrieval=retrieval(state_vector,var_to_retrieve,test_TBs,2,
                    ["20150314"],ar_plot_path)
Retrieval.build_K_reg()
# Retrieval coeffs
if len(synth_ar_dates)==1:
    coeff_file=var_to_retrieve+"_Retrieval_coeffs_20110317.csv"
    if take_haloac3_day:
        coeff_file=var_to_retrieve+"_Retrieval_coeffs_20220315.csv"
        print("HALO-AC3")
    m_est_heights=pd.read_csv(ar_plot_path+coeff_file,index_col=0)
else:
    m_est_heights=pd.read_csv(ar_plot_path+var_to_retrieve+"_Retrieval_coeffs_several_dates.csv",index_col=0)
# Create vertical q profile
var_retrieved=pd.DataFrame(data=np.nan, columns=["200","201"],index=regridded_test_era5["z"].values)
#print(Retrieval.K_reg)
if len(synth_ar_dates)>1:
    for z in range(regridded_test_era5["z"].shape[0]):
        var_retrieved.iloc[z,:]=m_est_heights.iloc[z,:].dot(Retrieval.K_reg.T)
else:
    for z in range(m_est_heights.shape[0]):
        m_coeffs=m_est_heights.iloc[z,:]
        var_retrieved.iloc[z,:]=m_coeffs.dot(Retrieval.K_reg.T)

if var_to_retrieve=="q":
    var_retrieved*=1000 

print(var_to_retrieve,":",var_retrieved)

ValueError: Must pass 2-d input. shape=(464, 44, 26)

In [ ]:
for idx in np.arange(200,202):
    if var_to_retrieve=="q":
        plt.plot(regridded_test_era5[var_to_retrieve][idx,:]*1000,regridded_test_era5["z"],color="lightblue")
        plt.plot(var_retrieved.iloc[:,idx-200],regridded_test_era5["z"],color="k",ls="--")
        plt.xlabel("Specific humidity q")
    elif var_to_retrieve=="log_q":
        plt.plot(np.exp(regridded_test_era5[var_to_retrieve][idx,:])*1000,regridded_test_era5["z"],color="lightblue")
        plt.plot(np.exp(var_retrieved.iloc[:,idx-200])*1000,regridded_test_era5["z"],color="k",ls="--")
        plt.xlabel("Specific humidity q")
    else:
        plt.plot(regridded_test_era5[var_to_retrieve][idx,:],regridded_test_era5["z"],color="lightblue")
        plt.plot(var_retrieved.iloc[:,idx-200],regridded_test_era5["z"],color="k",ls="--")
        plt.xlabel("Temperature K")

In [ ]:
if var_to_retrieve=="q":
    iwv=1/1.000*np.trapz(var_retrieved,axis=0,x=q_retrieved.index)/1000
    print(iwv)
    ##def calc_iwv_from_q(q_loc):
    ##    iwv_temporary=-1/g*np.trapz(q_loc,axis=0,x=pres_index)
    test_sim_cls.era5_ds["iwv"][200:202,0,0]            